In [1]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadwriteVal").getOrCreate() 
spark

archive_shutil.zip	       spark-warehouse
sample_data		       uw-madison-courses
spark-2.3.1-bin-hadoop2.7      uw-madison-courses.tar.xz
spark-2.3.1-bin-hadoop2.7.tgz  uw-madison-courses.zip


In [3]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

In [4]:
import shutil

shutil.make_archive('archive_shutil', format='zip', root_dir='uw-madison-courses')

'/content/archive_shutil.zip'

In [5]:
import os

path = "uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['subjects', 'schedules', 'courses', 'subject_memberships', 'rooms', 'sections']


In [6]:
subjects.show(4,False)

+----+------------------------------------+------------+
|code|name                                |abbreviation|
+----+------------------------------------+------------+
|908 |Soil Science                        |SOIL SCI    |
|350 |ENGLISH                             |ENGLISH     |
|351 |English as a Second Language        |ESL         |
|230 |Human Development and Family Studies|HDFS        |
+----+------------------------------------+------------+
only showing top 4 rows



In [7]:
schedules.show(4, False)

+------------------------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
|uuid                                |start_time|end_time|mon  |tues |wed  |thurs|fri  |sat  |sun  |
+------------------------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
|f2d66a4d-0c08-3b48-abf6-649fffd7ae90|-1        |-1      |false|false|false|false|false|false|false|
|875542a2-f786-34dd-933b-84a8af1aaaba|530       |580     |true |false|true |false|false|false|false|
|f41f1e4d-cb4f-3ded-b4b0-4a7c4da044e5|660       |710     |false|true |false|true |false|false|false|
|46da55a4-17f7-31a1-9492-fddb5af9cf13|800       |850     |false|true |false|true |false|false|false|
+------------------------------------+----------+--------+-----+-----+-----+-----+-----+-----+-----+
only showing top 4 rows



In [8]:
courses.show(4, False)

+------------------------------------+----------------------------------------------------------+------+
|uuid                                |name                                                      |number|
+------------------------------------+----------------------------------------------------------+------+
|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|Cooperative Education Program                             |1     |
|c070a84f-648f-351d-9499-5d0e30ad02cc|Cooperative Education/Co-op in Life Sciences Communication|1     |
|e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3|Cooperative Education Program                             |1     |
|8f63bde1-ff7f-3fe7-9901-862908bf134c|Workshop in Dance Activity                                |1     |
+------------------------------------+----------------------------------------------------------+------+
only showing top 4 rows



In [9]:
subject_memberships.show(4, False)

+------------+------------------------------------+
|subject_code|course_offering_uuid                |
+------------+------------------------------------+
|220         |344b3ebe-da7e-314c-83ed-9425269695fd|
|320         |344b3ebe-da7e-314c-83ed-9425269695fd|
|346         |344b3ebe-da7e-314c-83ed-9425269695fd|
|612         |344b3ebe-da7e-314c-83ed-9425269695fd|
+------------+------------------------------------+
only showing top 4 rows



In [10]:
step1 = courses.join(schedules, courses.uuid == schedules.uuid, how='left')
step1.limit(4).toPandas()

,uuid,name,number,uuid,start_time,end_time,mon,tues,wed,thurs,fri,sat,sun
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1,None,None,None,None,None,None,None,None,None,None
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1,None,None,None,None,None,None,None,None,None,None
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1,None,None,None,None,None,None,None,None,None,None
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1,None,None,None,None,None,None,None,None,None,None


In [11]:
valuesP = [('koala',1,'yes'),('caterpillar',2,'yes'),('deer',3,'yes'),('human',4,'yes')]
eats_plants = spark.createDataFrame(valuesP,['name','id','eats_plants'])

valuesM = [('shark',5,'yes'),('lion',6,'yes'),('tiger',7,'yes'),('human',4,'yes')]
eats_meat = spark.createDataFrame(valuesM,['name','id','eats_meat'])

eats_plants.show()

eats_meat.show()

+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

+-----+---+---------+
| name| id|eats_meat|
+-----+---+---------+
|shark|  5|      yes|
| lion|  6|      yes|
|tiger|  7|      yes|
|human|  4|      yes|
+-----+---+---------+



# **UNION**

In [12]:
df_concat = eats_plants.union(eats_meat)

In [13]:
df_concat.show()

+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
|      shark|  5|        yes|
|       lion|  6|        yes|
|      tiger|  7|        yes|
|      human|  4|        yes|
+-----------+---+-----------+



# **INNER JOIN**

In [14]:
inner_join = eats_plants.join(eats_meat, ["name","id"],"inner")
inner_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



# **LEFT JOIN**

In [15]:
left_join = eats_plants.join(eats_meat, ["name","id"],"left")
left_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



# **Conditional LEFT JOIN**

In [16]:
conditional_join = eats_plants.join(eats_meat, ["name","id"], how='left').filter(eats_meat.eats_meat=='yes')
conditional_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



# **Full OUTER JOIN**

In [17]:
full_outer_join = eats_plants.join(eats_meat, ["name","id"],how='full') 
full_outer_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      shark|  5|       null|      yes|
|      human|  4|        yes|      yes|
|      tiger|  7|       null|      yes|
|       lion|  6|       null|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+

